In [1]:
import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'

import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
from termcolor import colored



Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:

# Import twitter data
print(colored("Loading train and test data", "green"))

tr = pd.read_csv("/content/drive/My Drive/big.csv")
te = pd.read_csv("/content/drive/My Drive/twt_#ifc.csv")

tr.full_text=tr.full_text.astype(str)
te.full_text=te.full_text.astype(str)
print(colored("... \nData loaded", "green"))


Loading train and test data
... 
Data loaded


In [4]:

# Tokenization
print(colored("Tokenizing and padding data", "green"))
tokenizer = Tokenizer(num_words = 2000, split = ' ')
tokenizer.fit_on_texts(tr['full_text'].astype(str).values)
train_tweets = tokenizer.texts_to_sequences(tr['full_text'].astype(str).values)
max_len = max([len(i) for i in train_tweets])
train_tweets = pad_sequences(train_tweets, maxlen = max_len)
test_tweets = tokenizer.texts_to_sequences(te['full_text'].astype(str).values)
test_tweets = pad_sequences(test_tweets, maxlen = max_len)
print(colored("Tokenizing and padding complete", "yellow"))

Tokenizing and padding data
Tokenizing and padding complete


In [14]:
# Building the model
print(colored("Creating the Vanilla RNN model", "yellow"))
model = Sequential()
model.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(256,return_sequences = True))
model.add(LSTM(256,return_sequences = False))

model.add(Dense(3, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Creating the Vanilla RNN model
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 37, 128)           256000    
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 37, 128)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 37, 256)           394240    
_________________________________________________________________
lstm_11 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 771       
Total params: 1,176,323
Trainable params: 1,176,323
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Training the model
print(colored("Training the Vanilla RNN model", "green"))
history = model.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the Vanilla RNN model


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
20741/20741 [==============================] - 126s 6ms/step - loss: 0.8025 - accuracy: 0.6147
Epoch 2/3
20741/20741 [==============================] - 126s 6ms/step - loss: 0.5915 - accuracy: 0.7441
Epoch 3/3
20741/20741 [==============================] - 125s 6ms/step - loss: 0.5401 - accuracy: 0.7724


In [16]:
 print(colored("Testing the Vanilla RNN model", "green"))

loss, accuracy = model.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))

score, accuracy = model.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))



Testing the Vanilla RNN model
20741/20741 [==============================] - 31s 2ms/step
Train accuracy: 0.8015524744987488
1749/1749 [==============================] - 3s 2ms/step
Test accuracy: 0.7793024778366089
